# TP 2 - Bag of Words (BOW) & TF-IDF

L'objectif de ce TP est la construction d'un BOW pour la détection de SPAM sur une base de données de mails, puis l'implémentation de TF-IDF.

## Exercice 1 – BOW (Bag of Words)

In [ ]:
# 1. Importer les dépendances
import pandas as pd
import nltk
import numpy as np
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import WordNetLemmatizer
import re

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

# 2. Importer les données
# On utilise spam.csv qui semble correspondre à la structure attendue (v1, v2)
try:
    df = pd.read_csv('../data/spam.csv', encoding='latin-1')
except FileNotFoundError:
    print("Fichier non trouvé. Vérifiez le chemin.")

# Renommer les colonnes pour plus de clarté si nécessaire, ou garder v1/v2 comem dans le TP
# Le TP utilise v1 (label) et v2 (text)
df = df[['v1', 'v2']]
df.columns = ['label', 'text'] # Renommons pour clarté

print(df.head())

In [ ]:
# 3. Prétraitement des données
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

corpus = []
for i in range(len(df)):
    # Supprimer les caractères non alphabétiques
    review = re.sub('[^a-zA-Z]', ' ', df['text'][i])
    review = review.lower()
    review = review.split()
    
    # Lemmatisation et suppression des stopwords
    review = [lemmatizer.lemmatize(word) for word in review if not word in stop_words]
    review = ' '.join(review)
    corpus.append(review)

# Ajoutons le corpus nettoyé au dataframe pour vérification
df['cleaned_text'] = corpus
print(df.head())

In [ ]:
# 4. Implémenter BOW avec sklearn
from sklearn.feature_extraction.text import CountVectorizer

# max_features=2500 signifie qu'on ne garde que les 2500 mots les plus fréquents
# Avantage plus grand : capture plus de nuances, mais risque de sur-apprentissage et coût calculatoire
# Avantage plus petit : modèle plus simple, rapide, moins de bruit, mais perte d'info
cv = CountVectorizer(max_features=2500)
X = cv.fit_transform(corpus).toarray()

# 5. Transformation de la variable cible (label)
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y = le.fit_transform(df['label'])

print("Shape of X:", X.shape)
print("Shape of y:", y.shape)

## Exercice 2 – Modélisation

In [ ]:
# 1. Définition de la base d’apprentissage et test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 2. Apprentissage d’un modèle de classification (Teoreme de Bayes)
from sklearn.naive_bayes import MultinomialNB
model = MultinomialNB()
model.fit(X_train, y_train)

# 3. Prédiction et calcul des performances
y_pred = model.predict(X_test)

from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
acc = accuracy_score(y_test, y_pred)

print("Confusion Matrix:\n", cm)
print("Accuracy:", acc)

## Exercice 3 - TF-IDF

Utilisation de TF-IDF pour pondérer l'importance des termes.

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

# On repart des données brutes pour utiliser le pipeline complet
X_raw = df['text']
y_raw = df['label']

X_train_raw, X_test_raw, y_train_raw, y_test_raw = train_test_split(X_raw, y_raw, test_size=0.2, random_state=57)

# Définir une data pipeline
text_clf = Pipeline([
    ('vect', CountVectorizer(stop_words='english')), # Nettoyage et vectorisation
    ('tfidf', TfidfTransformer()), # TF-IDF
    ('clf', LogisticRegression()), # Classification
])

# Exécuter le pipeline
text_clf.fit(X_train_raw, y_train_raw)

# Prédiction
predicted = text_clf.predict(X_test_raw)

# Imprimer le rapport de performance
print("Accuracy:", accuracy_score(y_test_raw, predicted))
print("Confusion Matrix:\n", confusion_matrix(y_test_raw, predicted))

In [ ]:
# Validation croisée
from sklearn.model_selection import cross_val_score

scores = cross_val_score(text_clf, X_raw, y_raw, cv=5)
print("Cross-validation scores:", scores)
print("Mean score:", np.mean(scores))

In [ ]:
# Essayer quelques phrases
examples = [
    'Subscribe to my Youtube Channel!! :)',
    'Hi veronica, hope you are doing good',
    'Earn money for being online with 0 efforts! ...'
]
predictions = text_clf.predict(examples)

for text, label in zip(examples, predictions):
    print(f"'{text}' -> {label}")